In [16]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Sample dataset
data = '/Users/andreybalabin/Desktop/Sean_Gomez_Lab/klaeger.csv'
df = pd.read_csv(data)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("Kinase Inhibition Data"),
    dcc.RadioItems(
        id='radio-selection',
        options=[
            {'label': 'Select Compound', 'value': 'drug'},
            {'label': 'Select Kinase', 'value': 'gene'}
        ],
        value='drug',
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Dropdown(
        id='dropdown-select',
        multi=False
    ),
    html.Label("Select Concentration (M):"),
    dcc.Dropdown(
        id='dropdown-concentration',
        options=[{'label': conc, 'value': conc} for conc in df['concentration_M'].unique()],
        value=df['concentration_M'].unique()[0],
        multi=False
    ),
    dcc.Graph(id='kinase-inhibition-plot')
])

# Define callback to update the gene or drug dropdown based on radio button selection
@app.callback(
    Output('dropdown-select', 'options'),
    [Input('radio-selection', 'value')]
)
def update_dropdown(selected_value):
    if selected_value == 'drug':
        options = [{'label': 'Compound: ' + drug, 'value': drug} for drug in df['drug'].unique()]
    elif selected_value == 'gene':
        options = [{'label': 'Kinase: ' + gene, 'value': gene} for gene in df['gene_name'].unique()]
    else:
        options = []
    return options

# Define callback to update the plot based on dropdown selections
@app.callback(
    Output('kinase-inhibition-plot', 'figure'),
    [Input('dropdown-select', 'value'),
     Input('dropdown-concentration', 'value'),
     Input('radio-selection', 'value')]
)
def update_plot(selected_item, selected_concentration, selection_type):
    if selected_item is None:
        return {}
    
    if selection_type == 'drug':
        filtered_df = df[(df['drug'] == selected_item) & 
                         (df['concentration_M'] == selected_concentration) &
                         (df['relative_intensity'] < 1)]  # Exclude values of 1 or above
        x_label = 'gene_name'
        x_axis_label = f"Kinases Inhibited by {selected_item}"
        title_prefix = f"Kinase Inhibition for {selected_item}"
    elif selection_type == 'gene':
        filtered_df = df[(df['gene_name'] == selected_item) & 
                         (df['concentration_M'] == selected_concentration) &
                         (df['relative_intensity'] < 1)]  # Exclude values of 1 or above
        x_label = 'drug'
        x_axis_label = f"Compounds Inhibiting {selected_item}"
        title_prefix = f"Kinase Inhibition for {selected_item}"
    else:
        return {}
    
    # Sort the dataframe by 'relative_intensity'
    filtered_df = filtered_df.sort_values(by='relative_intensity')
    
    fig = px.bar(filtered_df, x=x_label, y='relative_intensity', 
                 color=x_label, hover_data=['drug', 'gene_name'])
    
    fig.update_layout(title=f"{title_prefix} at {selected_concentration} M (Excluding values >= 1)",
                      xaxis_title=x_axis_label,
                      yaxis_title='Inhibition')
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
